In [1]:
from pathlib import Path
from  pyprojroot import here
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException, StaleElementReferenceException

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from scrap_juicios import scrap_dependencias

from fake_useragent import UserAgent

In [2]:
# Define paths
root = here()
db = root/'data'

***
### Get lista de dependencias a buscar

In [3]:
# Load dependencias codes
db_depen = pd.read_csv(db/'raw/codigos_dependencias_satje.csv', header=None, names=['id_dependencia'])

# El Codigo de dependencia tiene 5 digitos. Lo convierto a string y anado un cero a los que tienen 4
db_depen['id_dependencia'] = db_depen['id_dependencia'].apply(lambda x: "0" + str(x) if len(str(x))==4 else str(x))
db_depen = db_depen.sort_values(by='id_dependencia').reset_index(drop=True)

***
### Webscrap the data

In [4]:
def obtener_codigos(restantes):

    # Start Selenium Session
    options = webdriver.FirefoxOptions()
    #options.headless = True # do not show browser window
    options.page_load_strategy = 'none' # Dont wait page to be loaded
    options.set_preference("general.useragent.override", UserAgent().random)

    # Start Driver
    gecko_path = Path.home()/'Documents/geckodriver.exe'
    driver = webdriver.Firefox(executable_path=gecko_path, options=options)
    url = 'http://consultas.funcionjudicial.gob.ec/informacionjudicial/public/informacion.jsf'
    driver.get(url)

    # Loop over faltantes
    for index, depnumber in enumerate(restantes['id_dependencia']):

        # Para capturar errores en el webscrap
        try:
            # Call webscraper
            respuesta = scrap_dependencias(driver, depnumber, delay=2)

            # If it worked
            if respuesta != False:
                restantes.loc[index, 'nombre'] = respuesta

            # Si no funciono, probamos cambiando year y caso
            else:
                # Probar con anios
                for yr in range(2019, 2022):
                    year = str(yr)

                    # Probar con secuenciales
                    for start in range(2, 7):
                        secuencial = (5 - len(str(start)))*'0' + str(start)
                        respuesta = scrap_dependencias(driver, depnumber, secuencial, year, delay=2)

                        if respuesta != False:
                            restantes.loc[index, 'nombre'] = respuesta
                            break
                    
                    if respuesta != False:
                        break

                    else:
                        restantes.loc[index, 'nombre'] = 'No se encuentra'

        except (ElementNotInteractableException, StaleElementReferenceException, PermissionError):
            driver.close()
            return {'estado': False, 'df': restantes}

    return {'estado': True, 'df': restantes}

In [5]:
# Load Datasets
df_dependencias = pd.read_csv(db/'raw/judicaturas/codes1.csv')
restantes = df_dependencias.loc[df_dependencias['nombre'].isna()].reset_index(drop = True)
completos = df_dependencias.loc[~df_dependencias['nombre'].isna()].copy()


# Primera iteracion
rest = obtener_codigos(restantes)

# Mientras haya faltantes
while rest['estado'] == False:
    dfdepen = pd.concat([completos, rest['df']], ignore_index=True).to_csv(db/'raw/judicaturas/codes1.csv', index=False)
    completos = pd.concat([completos, rest['df'].loc[~rest['df']['nombre'].isna()]], ignore_index=True)
    restantes = rest['df'].loc[rest['df']['nombre'].isna()].reset_index(drop=True).copy()
    rest = obtener_codigos(restantes)

else:
    dfdepen = pd.concat([completos, rest['df']], ignore_index=True).to_csv(db/'raw/judicaturas/codes1.csv', index=False)


***
### Get list of crimes

In [ ]:
crimenes = pd.read_excel(db/'raw/judicaturas/1123_Causas_delitos.xlsx', skiprows=6, usecols='B:F')
delitos = crimenes[['NOMBRE DELITO']].copy()
delitos.drop_duplicates(inplace=True)
delitos.dropna(inplace=True)
delitos.to_csv(db/'proc/lista_delitos.csv', index=False)